**Operations Research in Action &#x25aa; Fall 2024**

# Project 1 &ndash; Modeling &ndash; Part 1

## Review the available data

- Recall that the purpose of this project is to forecast the monthly demand for beer based on a variety of factors.

* Let's refresh our memory and take a look at the data:

In [1]:
import pandas as pd

In [2]:
demand_price_df = pd.read_excel(
    './data/TablesA3A4.xlsx',
    sheet_name='Table A3',
    skiprows=3,
    nrows=84,
    header=None,
    names=[
        'Year',
        'Month',
        'BeerConsumption',
        'AvgBeerPrice',
        'AvgRakiPrice',
        'AvgCannedSoftDrinkPrice',
        'AvgCannedBeerPrice',
        'AvgDraftBeerPrice'
    ]
)

demand_price_df.head()

,Year,Month,BeerConsumption,AvgBeerPrice,AvgRakiPrice,AvgCannedSoftDrinkPrice,AvgCannedBeerPrice,AvgDraftBeerPrice
0,1987,January,13621800,1338.424656,9715.74,2208.12,2671.750,971.574039
1,1987,February,14346944,1314.441618,9460.31,2150.07,2866.475,946.031197
2,1987,March,14912839,1267.542544,9122.77,2073.36,2764.199,912.276950
3,1987,April,16129850,1241.471640,8935.13,2030.71,2707.345,893.513173
4,1987,May,12140503,1183.481067,8517.76,1935.85,2580.882,851.776142


In [3]:
tourist_df = pd.read_excel(
    './data/TablesA3A4.xlsx',
    sheet_name='Table A4',
    skiprows=3,
    header=None,
    names=[
        'Year',
        'Month',
        'Czechoslovakia',
        'Germany',
        'UnitedKingdom',
        'UnitedStates',
        'France',
        'Others'
    ]
)

tourist_df.head()

,Year,Month,Czechoslovakia,Germany,UnitedKingdom,UnitedStates,France,Others
0,1987,January,290,28056,11380,25060,9246,35135
1,1987,February,253,29715,20630,21648,11941,38488
2,1987,March,478,67370,30506,30386,15810,53071
3,1987,April,1025,271147,70016,43137,94247,141639
4,1987,May,2776,405921,165298,69696,155816,225780


In [4]:
ramadan_days_df = pd.read_excel(
    './data/FromCase.xlsx',
    sheet_name='RamadanDays'
)

ramadan_days_df.head()

,Year,Month,RamadanDays
0,1987,January,0
1,1987,February,0
2,1987,March,0
3,1987,April,2
4,1987,May,28


In [5]:
beer_consumption_df = pd.read_excel(
    './data/FromCase.xlsx',
    sheet_name='DailyBeerConsumption',
    index_col='Country'
)

beer_consumption_df.head()

,AvgDailyBeerConsumptionPerCapita
Country,
Czechoslovakia,0.446575
Germany,0.378082
UnitedKingdom,0.273973
UnitedStates,0.235616
Others,0.184395


## Brainstorming a linear regression model

- As we discussed earlier, we can use __linear regression__ to forecast monthly beer demand.

- Recall the linear regression model:

$$Y = \beta_0 + \beta_1 X_1 + \dots + \beta_k X_k + \varepsilon \qquad \varepsilon \sim \text{iid } N(0, \sigma_{\varepsilon}^2)$$

- $Y$ is the __response variable__, of the variable that measures the outcome of interest.

- $X_1, \dots, X_k$ are the __explanatory variables__, the variables whose relationship to the response variable is being studied. 

* What should we use as the response variable?

_Write your notes here. Double-click to edit._

_Solution._ We would ideally use the monthly beer demand as the response variable, since that is what we want to predict. However, we don't have that data. We can use monthly beer consumption as a proxy instead, as long as we keep in mind that we've made the assumption that consumption is roughly equal to demand.

* What should we use as the explanatory variables? 

_Write your notes here. Double-click to edit._

_Solution._ There are a number of variables that might influence monthly beer demand: 

- average beer price 
- average Raki price 
- average canned soft drink price 
- average canned beer price 
- average draft beer price 
- tourist beer consumption
- year (to control for long-term effects)
- month (to control for seasonal effects)
- number of Ramadan days in the month

## Data wrangling

* To run a linear regression model with all of these variables, we need to create a new single data frame that contains all of them.

* To do this, we'll need to do some basic __data wrangling__.

* Let's merge all the tables together, matching rows based on `Year` and `Month`.

* We can accomplish this using the `.merge()`  method of a Pandas DataFrame.

* [Here is the documentation for `.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).

In [6]:
# Solution
all_df = (
    demand_price_df
    .merge(tourist_df, on=['Year', 'Month'], how='left')
    .merge(ramadan_days_df, on=['Year', 'Month'], how='left')
)

all_df.head()

,Year,Month,BeerConsumption,AvgBeerPrice,AvgRakiPrice,AvgCannedSoftDrinkPrice,AvgCannedBeerPrice,AvgDraftBeerPrice,Czechoslovakia,Germany,UnitedKingdom,UnitedStates,France,Others,RamadanDays
0,1987,January,13621800,1338.424656,9715.74,2208.12,2671.750,971.574039,290,28056,11380,25060,9246,35135,0
1,1987,February,14346944,1314.441618,9460.31,2150.07,2866.475,946.031197,253,29715,20630,21648,11941,38488,0
2,1987,March,14912839,1267.542544,9122.77,2073.36,2764.199,912.276950,478,67370,30506,30386,15810,53071,0
3,1987,April,16129850,1241.471640,8935.13,2030.71,2707.345,893.513173,1025,271147,70016,43137,94247,141639,2
4,1987,May,12140503,1183.481067,8517.76,1935.85,2580.882,851.776142,2776,405921,165298,69696,155816,225780,28


* Let's also add an estimate of the monthly tourist beer consumption to this DataFrame.

In [7]:
# Solution
all_df['TouristBeerConsumption'] = (
    all_df['Czechoslovakia'] * beer_consumption_df['AvgDailyBeerConsumptionPerCapita']['Czechoslovakia']
    + all_df['Germany'] * beer_consumption_df['AvgDailyBeerConsumptionPerCapita']['Germany'] 
    + all_df['UnitedKingdom'] * beer_consumption_df['AvgDailyBeerConsumptionPerCapita']['UnitedKingdom']
    + all_df['UnitedStates'] * beer_consumption_df['AvgDailyBeerConsumptionPerCapita']['UnitedStates'] 
    + all_df['France'] * beer_consumption_df['AvgDailyBeerConsumptionPerCapita']['France']
    + all_df['Others'] * beer_consumption_df['AvgDailyBeerConsumptionPerCapita']['Others']
)

all_df.head()

,Year,Month,BeerConsumption,AvgBeerPrice,AvgRakiPrice,AvgCannedSoftDrinkPrice,AvgCannedBeerPrice,AvgDraftBeerPrice,Czechoslovakia,Germany,UnitedKingdom,UnitedStates,France,Others,RamadanDays,TouristBeerConsumption
0,1987,January,13621800,1338.424656,9715.74,2208.12,2671.750,971.574039,290,28056,11380,25060,9246,35135,0,27226.000715
1,1987,February,14346944,1314.441618,9460.31,2150.07,2866.475,946.031197,253,29715,20630,21648,11941,38488,0,30473.275998
2,1987,March,14912839,1267.542544,9122.77,2073.36,2764.199,912.276950,478,67370,30506,30386,15810,53071,0,52677.449434
3,1987,April,16129850,1241.471640,8935.13,2030.71,2707.345,893.513173,1025,271147,70016,43137,94247,141639,2,168507.661703
4,1987,May,12140503,1183.481067,8517.76,1935.85,2580.882,851.776142,2776,405921,165298,69696,155816,225780,28,274701.483979


* Finally, let's output this DataFrame to a CSV file so that we can use it with R to do some regression modeling:

In [8]:
# Solution
all_df.to_csv('all.csv', index=False)